In [22]:
from evoVAE.utils.datasets import MSA_Dataset
import evoVAE.utils.seq_tools as st
from sklearn.model_selection import train_test_split
import pandas as pd

## Data loading and preprocessing

In [23]:

# define threshold for sequence similarity 
THETA = 0.2

# Read in the datasets and create train and validation sets 
alns: pd.DataFrame = st.read_aln_file("../data/alignments/tiny.aln")
train, val = train_test_split(alns, test_size=0.2)


# create one-hot encodings and calculate reweightings 

# TRAINING 
train_encodings, train_weights = st.encode_and_weight_seqs(train["sequence"],theta=THETA)
train_ids = train["id"].values # just the seq identifiers 
train_dataset = MSA_Dataset(train_encodings, train_weights, train_ids)

# VALIDATION
val_encodings, val_weights = st.encode_and_weight_seqs(val["sequence"], THETA)
val_ids = val["id"].values
val_dataset = MSA_Dataset(val_encodings, val_weights, val_ids)


Encoding the sequences and calculating weights
The sequence encoding tensor has size: torch.Size([2, 5, 21])
The sequence weight array has size: (2,)

Encoding the sequences and calculating weights
The sequence encoding tensor has size: torch.Size([1, 5, 21])
The sequence weight array has size: (1,)

